In [ ]:
import pandas as pd
import matplotlib as plt
from finx_ib_reports.download_trades import load_report

In [ ]:
fn = "data/flex_report_1648404346.xml"
report = load_report(fn)

In [ ]:

report.df("OpenPosition")

report.df("Trade")

In [ ]:
# fn = "~/Desktop/lsc-2022-q1.xlsx"
# df = pd.read_excel(fn)
# df.dtypes

# # df.Return = df.Return.astype(float)
# df[df.Return == "< -2000000"]

In [ ]:
# determine the underlying contract. Eg (ES, MES, MNQ, CL)
df["underlyingSymbol"] = df.Description.str.split("\s", expand=True).iloc[:, 0]

# combine Unrealized and Realized
df["totalPnL"] = df["Realized_P&L"] + df["Unrealized_P&L"]

def gen_mapper_dict():
    tag_def = {
        "Oil": ["CL", "MCL"],
        "EquityIndex": ["ES", "MES", "NQ", "MNQ"],
        "Corn": ["ZC", "YC"],
        "Rates": ["ZN"],
        "Gold": ["MGC", "GC"],
    }
    mapper_dict = {}
    for k,v in tag_def.items():
        for product in v:
            mapper_dict[product] = k
    return mapper_dict


# tag with business line
def categorize(row) -> str:
    md = gen_mapper_dict()
    return md[row['underlyingSymbol']]

In [ ]:
#
# End goal, group all instruments into the categories we trade and plot. 
# categories: Crude, EquityVol, Rates, Corn, and Gold
#
# underlying df
udf = df.groupby(["underlyingSymbol"]).agg({"totalPnL": "sum"}).reset_index()
udf['category'] = udf.apply(lambda row: categorize(row), axis=1)

# category df
cdf = x.groupby(["category"]).agg({"Contribution": "sum", "totalPnL": "sum"}).reset_index().set_index(["category"])
cdf["Percent"] = (cdf["totalPnL"] / cdf.totalPnL.sum())

# plot cdf
axes = cdf[["totalPnL", "Percent"]].plot.bar(rot=0, subplots=True, figsize=(10, 14))
axes[1].legend(loc=2)

# format dollar values
axes[0].get_yaxis().set_major_formatter(
    plt.ticker.FuncFormatter(lambda x, p: format(int(x), ',')))

# format percent values
axes[1].get_yaxis().set_major_formatter(
    plt.ticker.PercentFormatter())

cdf.reset_index().to_csv("lsc-2022-q1.csv")